In [1]:
'''
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""
'''

'\nimport os\nos.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152\nos.environ["CUDA_VISIBLE_DEVICES"] = ""\n'

In [2]:
import gym
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
#from keras.constraints import MaxNorm
import numpy as np
import tensorflow as tf
import gc

Using TensorFlow backend.


In [3]:
#Checking use of GPUs
from keras import backend as K
print(K.tensorflow_backend._get_available_gpus())
sess= K.get_session()

['/gpu:0']


In [4]:
#Initializing environment
env = gym.make('Humanoid-v2')
So = env.reset()
A = env.action_space.sample()

In [5]:
#Creating the model for policy
global policy_model
policy_model = Sequential()
policy_model.add(Dense(400,input_shape = np.append(So, A).shape, activation = 'sigmoid' ))
policy_model.add(Dropout(0.5))
policy_model.add(Dense(100, activation = 'sigmoid'))
policy_model.add(Dropout(0.5))
policy_model.add(Dense(30, activation = 'sigmoid'))
policy_model.add(Dense(1, activation= 'sigmoid'))

#adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
policy_model.compile(loss="mse", optimizer= "sgd")
print(policy_model.summary())
#policy_model.load_weights("policy.h5")

sess.run(tf.global_variables_initializer())

#Intializing eligibility traces for policy improvement
global policy_eligibility_traces
policy_eligibility_traces= [tf.zeros(shape = tensor.eval(session = sess).shape) for tensor in policy_model.trainable_weights]

#Setting learning hyperparameters 
global policy_alpha
policy_alpha = np.exp2(-2)
global policy_lambda
policy_lambda =  np.exp2(-2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 400)               157600    
_________________________________________________________________
dropout_1 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               40100     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 30)                3030      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 31        
Total params: 200,761
Trainable params: 200,761
Non-trainable params: 0
_________________________________________________________________
None

In [6]:
#Creating model for values 
global value_model
value_model = Sequential()
value_model.add(Dense(400,input_shape = So.shape, activation = 'relu' ))
value_model.add(Dropout(0.5))
value_model.add(Dense(100, activation = 'relu'))
value_model.add(Dropout(0.5))
value_model.add(Dense(30, activation = 'relu'))
value_model.add(Dense(1, activation= 'linear'))

#adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
value_model.compile(loss="mse", optimizer= "sgd")
print(value_model.summary())
#value_model.load_weights("value.h5")

sess.run(tf.global_variables_initializer())

#Intializing eligibility traces for value improvement
global value_eligibility_traces
value_eligibility_traces= [tf.zeros(shape = tensor.eval(session = sess).shape) for tensor in value_model.trainable_weights]

#Setting learning hyperparameters 
global value_alpha
value_alpha = np.exp2(-2)
global value_lambda
value_lambda = np.exp2(-2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 400)               150800    
_________________________________________________________________
dropout_3 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 100)               40100     
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 30)                3030      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 31        
Total params: 193,961
Trainable params: 193,961
Non-trainable params: 0
_________________________________________________________________
None

In [7]:
#Defining other hyperparameters

global average_reward
average_reward = 5
global etaa
etaa = 0.01

In [8]:
#Auxiliary functions for weights update

def set_weights(model, weights):  
    if(model=='value'):
        i=0
        for layer in value_model.layers:
            if 'dropout' not in layer.name:
                layer.set_weights([weights[i],weights[i+1]])
                i+=2
    elif(model=='policy'):
        i=0
        for layerz in policy_model.layers:
            if 'dropout' not in layerz.name:
                layerz.set_weights([weights[i],weights[i+1]])
                i+=2
    else:
        print('error')

def get_value_weights():
    weights =[]
    for w in value_model.layers:
        weights+= w.weights
    return weights

def get_policy_weights():
    weights =[]
    for w in policy_model.layers:
        weights+= w.weights
    return weights

In [9]:
#Update weights
def update_weights(previous_state, action , reward, state,  terminal):
    
    #Compute rewards
    global average_reward
    delta_rewards = reward - average_reward + value_model.predict(np.array([state]))[0][0] - value_model.predict(np.array([previous_state]))[0][0]    
    if(terminal):
        delta_rewards = reward - average_reward + 0 - value_model.predict(np.array([previous_state]))[0][0]
    average_reward = average_reward + etaa*delta_rewards
    
    #Compute value updates (eligibility traces and weights)
    global value_eligibility_traces
    value_gradients = value_model.optimizer.get_gradients(value_model.output, value_model.trainable_weights)
    value_eligibility_traces_op = [tf.add(tf.multiply(value_eligibility_traces[i], tf.constant(value_lambda,dtype= tf.float32)),value_gradients[i] ) for i in range(len(value_eligibility_traces))] 
    value_eligibility_traces = [tf.convert_to_tensor(x) for x in sess.run(value_eligibility_traces_op,feed_dict={value_model.input:np.array([previous_state])})]
    
    
    print(sess.run(value_gradients, feed_dict={value_model.input:np.array([So])}))
    
    value_weights = get_value_weights()
    value_weights_op =  [tf.add(value_weights[i], tf.multiply(tf.constant(value_alpha*delta_rewards,dtype=tf.float32), value_eligibility_traces[i])) for i in range(len(value_weights))]
    set_weights('value', sess.run(value_weights_op))
    
    #Compute policy updates (eligibility traces and weights)
    global policy_eligibility_traces
    policy_gradients = policy_model.optimizer.get_gradients(policy_model.output, policy_model.trainable_weights)
    policy_value = policy_model.predict(np.array([np.append(previous_state, action)]))[0][0]
    policy_value = 1/ (policy_value+1)
    policy_eligibility_traces_op = [tf.add(tf.multiply(tf.constant(policy_lambda,dtype=tf.float32), policy_eligibility_traces[i]), tf.multiply(tf.constant(policy_value, dtype= tf.float32), policy_gradients[i])) for i in range(len(policy_gradients))]
    policy_eligibility_traces = [tf.convert_to_tensor(x) for x in sess.run(policy_eligibility_traces_op, feed_dict={policy_model.input:np.array([np.append(previous_state,action)])})]
    policy_weights = get_policy_weights()
    policy_weights_op =  [tf.add(policy_weights[i], tf.multiply( policy_eligibility_traces[i],tf.constant(policy_alpha*delta_rewards,dtype=tf.float32))) for i in range(len(policy_weights))]
    set_weights('policy', sess.run(policy_weights_op))

In [10]:
#Creating model for remembering preferences over each state
number_of_preferences =10
global preference_model
preference_model = Sequential()
preference_model.add(Dense(400,input_shape = So.shape, activation = 'relu' ))
preference_model.add(Dropout(0.5))
preference_model.add(Dense(100, activation = 'relu'))
preference_model.add(Dropout(0.5))
preference_model.add(Dense(30, activation = 'relu'))
preference_model.add(Dense(A.shape[0]*number_of_preferences, activation= 'linear'))

adam = Adam(lr=0.4, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
preference_model.compile(loss="mse", optimizer=adam)
print(preference_model.summary())
#preference_model.load_weights('preference.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 400)               150800    
_________________________________________________________________
dropout_5 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 100)               40100     
_________________________________________________________________
dropout_6 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 30)                3030      
_________________________________________________________________
dense_12 (Dense)             (None, 170)               5270      
Total params: 199,200
Trainable params: 199,200
Non-trainable params: 0
_________________________________________________________________
None

In [ ]:
#Selecting an action
def select_action(state):
    
    number_of_new_actions = 10
    
    predicted_preferences = preference_model.predict(np.array([state]))[0].reshape((number_of_preferences,A.shape[0]))
    new_actions = env.action_space.sample().reshape(1, A.shape[0])
    
    for _ in range(number_of_new_actions-1):
        new_actions= np.concatenate((new_actions,env.action_space.sample().reshape(1, A.shape[0])), axis=0 )
    
    all_actions = np.concatenate((predicted_preferences, new_actions), axis=0)
    all_actionz = np.concatenate((np.tile(So, (all_actions.shape[0], 1)), all_actions),axis=1)
    policy_preferences = policy_model.predict(all_actionz)
    
    actions_list = []
    for i in range(all_actions.shape[0]):
        actions_list.append((all_actions[i], policy_preferences[i][0]))
    actions_list = sorted(actions_list, key = lambda x: x[1], reverse = True)
    
    probabilities = sess.run(tf.contrib.layers.softmax(tf.constant([[x[1] for x in actions_list]])))[0]
    p = np.random.rand()
    i=0
    for j in range(probabilities.shape[0]):
        p-=probabilities[j]
        if(p<0):
            i=j
    
    best_actions = np.array([x[0] for x in actions_list[0:number_of_preferences]]).flatten()
    preference_model.fit(np.array([state]), np.array([best_actions]), verbose=0)
    return actions_list[j][0]
        
    

In [ ]:
#Main loop
S = env.reset()
#action_count=0
episode_count=0
save= True
while(True):
    
    #renders environment 
    #env.render()
    
    
    #Selects action according to stochastic policy
    action = select_action(S)
    #action_count+=1
    
    #Takes action 
    S1, reward, done, info = env.step(action)
    
    
    #Updates weights
    if(np.random.randint(6)%3==0):
        update_weights(S , action, reward, S1 , done)
    
    S = S1
    
    if(done):
        
        print('We are now at '+str(episode_count))
        #gc.collect()
        
        #Resets episode
        S = env.reset()
        #action_count=0
        episode_count+=1
        if(episode_count%5==0):
            
            if(save):
                print('saving models')
                print('average reward '+ str(average_reward))
                preference_model.save('preference.h5')
                policy_model.save('policy.h5')
                value_model.save('value.h5')

        
        #Resets eligibility traces
        global value_eligibility_traces, policy_eligibility_traces
        policy_eligibility_traces= [tf.zeros(shape = tensor.eval(session = sess).shape) for tensor in policy_model.trainable_weights]
        value_eligibility_traces= [tf.zeros(shape = tensor.eval(session = sess).shape) for tensor in value_model.trainable_weights]
    

[array([[  1.19928665e-01,   0.00000000e+00,   4.41014580e-02, ...,
         -6.03312179e-02,   0.00000000e+00,   3.55482325e-02],
       [  8.58378112e-02,   0.00000000e+00,   3.15652005e-02, ...,
         -4.31814976e-02,   0.00000000e+00,   2.54433099e-02],
       [  2.25888900e-04,   0.00000000e+00,   8.30662902e-05, ...,
         -1.13635484e-04,   0.00000000e+00,   6.69560613e-05],
       ..., 
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -0.00000000e+00,   0.00000000e+00,   0.00000000e+00]], dtype=float32), array([ 0.08584382,  0.        ,  0.03156741,  0.        , -0.06504609,
        0.09862997,  0.01367242,  0.00353395,  0.06738153,  0.        ,
        0.        ,  0.        ,  0.        ,

[array([[ 37744.5546875 ,      0.        ,      0.        , ...,
             0.        ,      0.        ,      0.        ],
       [ 27015.30664062,      0.        ,      0.        , ...,
             0.        ,      0.        ,      0.        ],
       [    71.09288788,      0.        ,      0.        , ...,
             0.        ,      0.        ,      0.        ],
       ..., 
       [     0.        ,      0.        ,      0.        , ...,
             0.        ,      0.        ,      0.        ],
       [     0.        ,      0.        ,      0.        , ...,
             0.        ,      0.        ,      0.        ],
       [     0.        ,      0.        ,      0.        , ...,
             0.        ,      0.        ,      0.        ]], dtype=float32), array([  2.70171973e+04,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,  -2.59198975e+02,   5.53831940e+01,
         0.00000000e+00,   0.00000000e+00,

C:\Users\Fernando\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: RuntimeWarning: invalid value encountered in double_scalars


[array([[ inf,   0.,  inf, ...,   0.,  inf,   0.],
       [ inf,   0.,  inf, ...,   0.,  inf,   0.],
       [ inf,   0.,  inf, ...,   0.,  inf,   0.],
       ..., 
       [ nan,   0.,  nan, ...,   0.,  nan,   0.],
       [ nan,   0.,  nan, ...,   0.,  nan,   0.],
       [ nan,   0.,  nan, ...,   0.,  nan,   0.]], dtype=float32), array([             inf,   0.00000000e+00,              inf,
                    inf,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,              inf,
         0.00000000e+00,   0.00000000e+00,              inf,
                    inf,   0.00000000e+00,              inf,
         0.00000000e+00,   0.00000000e+00,              inf,
         0.00000000e+00,              inf,  -1.24601332e+30,
         0.00000000e+00,   0.00000000e+00,   5.14219771e+29,
         0.00000000e+00,   0.00000000e+00,  -3.14254101e+30,
         1.84470792e+30,   2.67828448e+30,   0.00000000e+00,
                    inf,              inf,              inf

[array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32), array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 

...

[array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32), array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 

[array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32), array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 

[array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32), array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 

[array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32), array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 

[array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32), array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 